In [10]:
import pickle
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Lambda
from keras.callbacks import ModelCheckpoint

In [2]:
data_dict = pickle.load(open('../data/shakespeare/processed.pickle', 'rb'))

In [3]:
X = data_dict['X']
Y = data_dict['Y']
dataX = data_dict['dataX']
dataY = data_dict['dataY']
char_to_idx = data_dict['char_to_idx']
idx_to_char = data_dict['idx_to_char']

# Constants
m, n_timesteps, _ = X.shape
_, n_chars = Y.shape
n_a = 128

In [4]:
print('X shape:', X.shape)
print('Y shape:', Y.shape)
print('Num. Timesteps:', n_timesteps)
print('Num. Unique Chars:', n_chars)

X shape: (13823, 140, 1)
Y shape: (13823, 35)
Num. Timesteps: 140
Num. Unique Chars: 35


In [5]:
def get_model(n_timesteps, n_features, n_a):
    x0 = Input(shape=(n_timesteps,1), name='input')
    
    X = LSTM(n_a, return_sequences=True)(x0)
    X = LSTM(n_a)(X)
    out = Dense(n_features, activation='softmax')(X)
    model = Model(x0, out)
    
    return model

In [6]:
model = get_model(n_timesteps, n_chars, n_a)

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [14]:
file_path = '../data/shakespeare/weights.{epoch:02d}-{loss:.2f}.hdf5'
checkpoint_callback = ModelCheckpoint(file_path, 
                                      monitor='loss',
                                      verbose=1, 
                                      save_best_only=True,
                                      mode='min')

In [15]:
a_initial = np.zeros((n_timesteps, n_a))
c_initial = np.zeros((n_timesteps, n_a))

In [16]:
history = model.fit(X, Y, 
                    epochs=20, 
                    batch_size=64, 
                    callbacks=[checkpoint_callback])

Epoch 1/20
13823/13823 [==============================] - 72s 5ms/step - loss: 2.9915

Epoch 00001: loss improved from inf to 2.99152, saving model to ../data/shakespeare/weights.01-2.99.hdf5
Epoch 2/20
13823/13823 [==============================] - 73s 5ms/step - loss: 2.9617

Epoch 00002: loss improved from 2.99152 to 2.96169, saving model to ../data/shakespeare/weights.02-2.96.hdf5
Epoch 3/20
13823/13823 [==============================] - 73s 5ms/step - loss: 2.8868

Epoch 00003: loss improved from 2.96169 to 2.88677, saving model to ../data/shakespeare/weights.03-2.89.hdf5
Epoch 4/20
13823/13823 [==============================] - 73s 5ms/step - loss: 2.8250

Epoch 00004: loss improved from 2.88677 to 2.82504, saving model to ../data/shakespeare/weights.04-2.83.hdf5
Epoch 5/20
13823/13823 [==============================] - 72s 5ms/step - loss: 2.7789

Epoch 00005: loss improved from 2.82504 to 2.77885, saving model to ../data/shakespeare/weights.05-2.78.hdf5
Epoch 6/20
13823/13823 [

In [ ]:
# Generate Text
# Given seed sentence - 140 characters - predict next character
# For every predicted char, add to predictions = []

def reshape_input(original_input):
    return np.reshape(original_input, (1, n_timesteps, 1))

def get_p_idx(p):
    flattened = np.ndarray.flatten(np.array(p))
    return np.random.choice([i for i in range(n_chars)], 
                            p = flattened)

def generate_text(seed_input, model, num_chars_to_generate=140):
    if len(seed_input) < 140:
        raise Exception('Seed_input must be at least 140 characters')
    curr_input = seed_input.lower()
    curr_input = list(curr_input)
    curr_input = curr_input[:n_timesteps] # first 140 chars
    
    curr_input = [char_to_idx[c] for c in curr_input]
    # Normalize
    curr_input = np.array(curr_input) / n_chars
    predictions = []
    
    for i in range(num_chars_to_generate):
        p = model.predict(reshape_input(curr_input))
        p_idx = get_p_idx(p)
        predictions.append(idx_to_char[p_idx])
        curr_input = np.append(curr_input[1:], p_idx / n_chars)
    
    return ''.join(predictions)

In [ ]:
seed_input = "Haply that name of 'chaste' unhappily set This bateless edge on his keen appetite; When Collatine unwisely did not let To praise the clear unmatched red and white Which triumph'd in that sky of his delight, Where mortal stars, as bright as heaven's beauties, With pure aspects did him peculiar duties."

In [ ]:
generate_text(seed_input, model)